## Opening Data: ##

In [97]:
import json
import ast
import pandas
from collections import defaultdict
import numpy as np

In [98]:
female_reviews = []
count_errors = 0
with open('review_buckets_female.txt') as data_file:    
    for line in data_file:
        try:
            female_reviews.append(ast.literal_eval(line))
        except:
            line = "{'"+line[2:]
            try:
                female_reviews.append(ast.literal_eval(line))
            except:
                count_errors+=1
print(str(len(female_reviews))+" female reviews read with "+str(count_errors)+ " error(s).")

64366 female reviews read with 0 error(s).


In [99]:
# fixing key issues
for i in range(len(female_reviews)):
    review = female_reviews[i]
    for key in review.keys():
        if 'movie_id' in key:
            review['movie_id'] = review[key]
        if 'icon' in key:
            review['icon'] = review[key]
    female_reviews[i] = review

In [100]:
male_reviews = []
count_errors = 0
with open('review_buckets_male.txt') as data_file:    
    for line in data_file:
        try:
            male_reviews.append(ast.literal_eval(line))
        except:
            count_errors+=1
print(str(len(male_reviews))+" male reviews read with "+str(count_errors)+ " error(s).")

64366 male reviews read with 0 error(s).


In [101]:
# fixing key issues
for i in range(len(male_reviews)):
    review = male_reviews[i]
    for key in review.keys():
        if 'movie_id' in key:
            review['movie_id'] = review[key]
        if 'icon' in key:
            review['icon'] = review[key]
    male_reviews[i] = review

In [102]:
movie_data = pandas.read_csv('movie_data_cleaned.csv')
movie_titles = [repr(x).split("\\", 1)[0].lower()[1:] for x in list(movie_data['movie_title'])]

In [103]:
male_review_movie_list = []
male_movie_to_review = defaultdict(list)
for i in male_reviews:
    if not str.isdigit(i['movie_id']):
        male_review_movie_list.append(i['movie_id'])
        male_movie_to_review[i["movie_id"]].append(i)
male_review_movie_list = set(male_review_movie_list)

In [104]:
female_review_movie_list = []
female_movie_to_review = defaultdict(list)
for i in female_reviews:
    if not str.isdigit(i['movie_id']):
        female_review_movie_list.append(i['movie_id'])
        female_movie_to_review[i["movie_id"]].append(i)
female_review_movie_list = set(female_review_movie_list)

In [111]:
i=0
for key in male_movie_to_review:
    if (i==0):
        print(key)
        print(male_movie_to_review[key][0])
    i=i+1

from_hell
{'movie_id': 'from_hell', 'name': 'Eugene Novikov', 'gender': 'm', 'review': 'The paper-thin plot is fruitlessly camouflaged by a deluge of camera tricks that would make even Oliver Stone blush, and the movie itself buckles under the pressure.', 'source': 'Film Blather', 'link': 'http://filmblather.com/films/fromhell/', 'date': 'December 6, 2001', 'id': 'eugene-novikov', 'icon': 'rotten'}


In [112]:
review_movie_id_to_imdb_id = {}

overlap_male_imdb = []
overlap_male_rt = []
for i in movie_titles:
    if i.replace(" ", "_") in male_review_movie_list:
        overlap_male.append(i)
        review_movie_id_to_imdb_id[i.replace(" ", "_")] = i
        overlap_male_rt.append(i.replace(" ", "_"))

overlap_female_imdb = []
overlap_female_rt = []
for i in movie_titles:
    if i.replace(" ", "_") in female_review_movie_list:
        overlap_female_imdb.append(i)
        overlap_female_rt.append(i.replace(" ", "_"))
        
overlap_intr = set(overlap_male) & set(overlap_female)
overlap_union = set(overlap_male) | set(overlap_female)

In [113]:
print(len(overlap_intr))
print(len(overlap_union))
print(len(set(overlap_female_rt)))

885
885
0


In [115]:
overlap_male_rt

[]

In [48]:
rt_movies = list(set(overlap_female_rt))
movie_to_idx = {}
for i in range(len(rt_movies)):
    movie_to_idx[rt_movies[i]] = i

In [58]:
reviewers = defaultdict(list)
for i in male_movie_to_review:
    for j in male_movie_to_review[i]:
        if (j["movie_id"] in rt_movies):
            j["gender"] = "m"
            reviewers[j["name"]].append(j)
for i in female_movie_to_review:
    for j in female_movie_to_review[i]:
        if (j["movie_id"] in rt_movies):
            j["gender"] = "f"
            reviewers[j["name"]].append(j)
reviewers_lst = []
for i in reviewers:
    reviewers_lst.append(i)

In [59]:
num_reviewers = len(set(reviewers))
reviewer_to_idx = {}
for i in range(len(reviewers_lst)):
    reviewer_to_idx[reviewers_lst[i]]=i
print(len(reviewers_lst))
print(num_reviewers)

2563
2563


In [71]:
review_matrix = np.zeros((num_reviewers,len(rt_movies)))
for i in reviewers:
    row = reviewer_to_idx[i]
    for j in reviewers[i]:
        col = movie_to_idx[j["movie_id"]]
        if (j["icon"] == "rotten"):
            review_matrix[row][col] = -1
        else:
            review_matrix[row][col] = 1

In [75]:
# review_matrix is reviewer by movie
count_reviews = np.zeros(num_reviewers)
for i in range(num_reviewers):
    for j in range(len(rt_movies)):
        if (review_matrix[i][j]==-1 or review_matrix[i][j]==1):
            count_reviews[i]+=1
print(count_reviews)

[ 14.   1.  57. ...,   1.   1.   2.]


In [92]:
review_row_idx_to_imdb_row = np.zeros(len(rt_movies))
count = 0
for i in range(len(rt_movies)):
    idx = movie_to_idx[rt_movies[i]]
    for j in range(len(movie_titles)):
        if movie_titles[j].replace(" ", "_") == rt_movies[i]:
            review_row_idx_to_imdb_row[i]=int(j)

In [93]:
review_row_idx_to_imdb_row

array([  2.34000000e+02,   1.44900000e+03,   2.72000000e+02,
         2.77300000e+03,   7.07000000e+02,   3.92200000e+03,
         1.46400000e+03,   8.42000000e+02,   2.67800000e+03,
         5.51000000e+02,   1.63700000e+03,   4.50000000e+01,
         2.86100000e+03,   7.34000000e+02,   9.06000000e+02,
         1.46000000e+03,   2.55300000e+03,   1.77000000e+02,
         1.64800000e+03,   1.89000000e+02,   6.49000000e+02,
         2.83800000e+03,   2.28400000e+03,   4.69400000e+03,
         1.65000000e+02,   1.23600000e+03,   7.84000000e+02,
         2.05600000e+03,   4.13000000e+02,   4.07000000e+02,
         4.12000000e+02,   2.34100000e+03,   1.56100000e+03,
         3.31000000e+02,   2.09000000e+02,   2.01000000e+02,
         2.41100000e+03,   2.30000000e+03,   6.95000000e+02,
         2.93000000e+02,   3.05300000e+03,   3.18600000e+03,
         2.19700000e+03,   4.05800000e+03,   1.56600000e+03,
         2.29200000e+03,   2.70100000e+03,   1.10000000e+01,
         3.26000000e+02,

In [94]:
np.shape(review_row_idx_to_imdb_row)

(885,)

In [95]:
np.shape(review_matrix)

(2563, 885)

In [96]:
np.shape(movie_data)

(5043, 28)